In [1]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime

import pandas as pd

from key_value.token import INFLUXDB_TOKEN as token

In [2]:
org = "boonsak.s@ku.th"
url = "https://europe-west1-1.gcp.cloud2.influxdata.com"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [9]:
query_api = client.query_api()
# |> range(start: -30d)

query = """from(bucket: "Flight_data")
 |> range(start: -30d, stop:-21d)
 |> filter(fn: (r) => r._measurement == "flight_id")

 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
 |> sort(columns: ["_time"], desc: false)
 """
tables = query_api.query_data_frame(query, org="boonsak.s@ku.th")
tables


,result,table,_time,_start,_stop,_measurement,altitude,groundspeed,heading,latitude,longitude
0,_result,0,2022-11-14 17:23:18+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,10.0,180.0,209.0,13.889838,100.591895
1,_result,1,2022-11-14 17:23:19+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,10.25,181.0,209.0,13.889081,100.591459
2,_result,2,2022-11-14 17:23:20+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,10.5,182.0,209.0,13.888324,100.591022
3,_result,3,2022-11-14 17:23:21+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,10.75,183.0,209.0,13.887567,100.590586
4,_result,4,2022-11-14 17:23:22+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,11.0,184.0,209.0,13.886810,100.590150
...,...,...,...,...,...,...,...,...,...,...,...
256,_result,256,2022-11-14 17:27:22+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,95.4,282.1333333333333,152.86666666666667,13.699118,100.438155
257,_result,257,2022-11-14 17:27:23+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,95.8,282.1,152.4,13.697906,100.438736
258,_result,258,2022-11-14 17:27:24+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,96.2,282.06666666666666,151.93333333333334,13.696694,100.439317
259,_result,259,2022-11-14 17:27:25+00:00,2022-11-06 05:24:59.967871+00:00,2022-11-15 05:24:59.967871+00:00,flight_id,96.6,282.03333333333336,151.46666666666667,13.695482,100.439899


In [85]:
tables.drop(columns=['result','table','_start','_stop','_measurement'], inplace=True)
tables.sort_values(by='_time', inplace=True)
tables.reset_index(drop=True, inplace=True)
tables

,_time,altitude,groundspeed,heading,latitude,longitude
0,2022-11-14 17:23:06+00:00,7.0,168.0,209.0,13.898920,100.597130
1,2022-11-14 17:23:07+00:00,7.25,169.0,209.0,13.898163,100.596694
2,2022-11-14 17:23:08+00:00,7.5,170.0,209.0,13.897406,100.596257
3,2022-11-14 17:23:09+00:00,7.75,171.0,209.0,13.896649,100.595821
4,2022-11-14 17:23:10+00:00,8.0,172.0,209.0,13.895893,100.595385
...,...,...,...,...,...,...
256,2022-11-14 17:27:22+00:00,95.4,282.1333333333333,152.86666666666667,13.699118,100.438155
257,2022-11-14 17:27:23+00:00,95.8,282.1,152.4,13.697906,100.438736
258,2022-11-14 17:27:24+00:00,96.2,282.06666666666666,151.93333333333334,13.696694,100.439317
259,2022-11-14 17:27:25+00:00,96.6,282.03333333333336,151.46666666666667,13.695482,100.439899


In [76]:
# tables._time.unique()
# tables[tables.groundspeed ==tables.groundspeed.min()]
# tables.groundspeed.min()
tables

,_time,altitude,groundspeed,heading,latitude,longitude
0,2022-11-14 17:23:18+00:00,10.0,180.0,209.0,13.889838,100.591895
1,2022-11-14 17:23:19+00:00,10.25,181.0,209.0,13.889081,100.591459
2,2022-11-14 17:23:20+00:00,10.5,182.0,209.0,13.888324,100.591022
3,2022-11-14 17:23:21+00:00,10.75,183.0,209.0,13.887567,100.590586
4,2022-11-14 17:23:22+00:00,11.0,184.0,209.0,13.886810,100.590150
...,...,...,...,...,...,...
256,2022-11-14 17:27:22+00:00,95.4,282.1333333333333,152.86666666666667,13.699118,100.438155
257,2022-11-14 17:27:23+00:00,95.8,282.1,152.4,13.697906,100.438736
258,2022-11-14 17:27:24+00:00,96.2,282.06666666666666,151.93333333333334,13.696694,100.439317
259,2022-11-14 17:27:25+00:00,96.6,282.03333333333336,151.46666666666667,13.695482,100.439899


In [77]:
tables.sort_values(by='_time')


,_time,altitude,groundspeed,heading,latitude,longitude
180,2022-11-14 17:23:06+00:00,7.0,168.0,209.0,13.898920,100.597130
181,2022-11-14 17:23:07+00:00,7.25,169.0,209.0,13.898163,100.596694
182,2022-11-14 17:23:08+00:00,7.5,170.0,209.0,13.897406,100.596257
183,2022-11-14 17:23:09+00:00,7.75,171.0,209.0,13.896649,100.595821
209,2022-11-14 17:23:10+00:00,8.0,172.0,209.0,13.895893,100.595385
...,...,...,...,...,...,...
256,2022-11-14 17:27:22+00:00,95.4,282.1333333333333,152.86666666666667,13.699118,100.438155
257,2022-11-14 17:27:23+00:00,95.8,282.1,152.4,13.697906,100.438736
258,2022-11-14 17:27:24+00:00,96.2,282.06666666666666,151.93333333333334,13.696694,100.439317
259,2022-11-14 17:27:25+00:00,96.6,282.03333333333336,151.46666666666667,13.695482,100.439899


In [68]:
df = pd.read_csv('../Preprocess/data/Sample_data3.csv')
# df.groundspeed.min()
df[df.groundspeed ==df.groundspeed.min()]

,Unnamed: 0,timestamp,altitude,groundspeed,heading,latitude,longitude,segment
0,0,2022-10-04 17:23:06,7.0,168.0,209.0,13.89892,100.59713,1


# write api


In [61]:
bucket="Flight_data"

write_api = client.write_api(write_options=SYNCHRONOUS)
points = []
test_time =[]

for index, row in df.iterrows():
  date_format = '%Y-%m-%d %H:%M:%S'
  dtObj = datetime.strptime(row.timestamp, date_format)
  dtObj = dtObj+ pd.DateOffset(months=1, days=10)
  test_time.append(dtObj)
  point = (
    Point("flight_id")
    .tag("altitude",row.altitude)
    .tag("groundspeed",row.groundspeed)
    .tag("heading",row.heading)
    .field("latitude",row.latitude)
    .field("longitude",row.longitude)
    # .field("coord", (row.longitude, row.latitude))
    .time(dtObj)
  )
  points.append(point)
  # print()
# print(points)
write_api.write(bucket=bucket, org="boonsak.s@ku.th", record=points, write_precision='s')
# time.sleep(1) # separate points by 1 second

In [ ]:
# date_format = '%Y-%m-%d %H:%M:%S'
# dtObj = datetime.strptime(row.timestamp, date_format)
# dtObj = dtObj+ pd.DateOffset(months=1, days=10)

Timestamp('2022-11-14 17:27:26')

In [62]:
test_time

[Timestamp('2022-11-14 17:23:06'),
 Timestamp('2022-11-14 17:23:07'),
 Timestamp('2022-11-14 17:23:08'),
 Timestamp('2022-11-14 17:23:09'),
 Timestamp('2022-11-14 17:23:10'),
 Timestamp('2022-11-14 17:23:11'),
 Timestamp('2022-11-14 17:23:12'),
 Timestamp('2022-11-14 17:23:13'),
 Timestamp('2022-11-14 17:23:14'),
 Timestamp('2022-11-14 17:23:15'),
 Timestamp('2022-11-14 17:23:16'),
 Timestamp('2022-11-14 17:23:17'),
 Timestamp('2022-11-14 17:23:18'),
 Timestamp('2022-11-14 17:23:19'),
 Timestamp('2022-11-14 17:23:20'),
 Timestamp('2022-11-14 17:23:21'),
 Timestamp('2022-11-14 17:23:22'),
 Timestamp('2022-11-14 17:23:23'),
 Timestamp('2022-11-14 17:23:24'),
 Timestamp('2022-11-14 17:23:25'),
 Timestamp('2022-11-14 17:23:26'),
 Timestamp('2022-11-14 17:23:27'),
 Timestamp('2022-11-14 17:23:28'),
 Timestamp('2022-11-14 17:23:29'),
 Timestamp('2022-11-14 17:23:30'),
 Timestamp('2022-11-14 17:23:31'),
 Timestamp('2022-11-14 17:23:32'),
 Timestamp('2022-11-14 17:23:33'),
 Timestamp('2022-11-